*Technology transfer for rapid family forest assessments and stewardship planning*

# Generate forest indicators from Lidar data

This notebook, in conjunction with the Forest Mapping Metrics - GIS notebook (which should be run before this notebook), generates wide variety forest indicators that will be used in predicting forest type maps for a sample area within Clackamas County Oregon.  This is a small subset of our larger study region and is intended for methodological demonstration. 

Airborne lidar data, sourced from the __[Oregon Department of Geology and Mineral Industries (DOGAMI)](http://www.oregongeology.org/lidar/)__, has shown exceptional potential for a wide range of forest resource assessments and will serve as a primary resource in this analysis. We will utilize lidar data in two focal areas, the Portland Metro 2014 LiDAR acquisition footprint, and a southern Oregon area running from east of Ashland, OR over to Klamath Falls with lidar acquisitions ranging from 2004-2017.  This subarea analysis is located entirely within the Portland Metro acquistion. 

In [1]:
wd <- getwd()
points <- file.path(wd, "Data/Lidar/source/POINTS/ALL_POINTS")
dem <- file.path(wd, "Data/Lidar/source/Bare_Earth")
outDir <- file.path(wd, "Data/Lidar/output")
plots <- file.path(wd, "Data/FIA/FIA_plots.shp")

In [2]:
#generate list of .las files to be used as input for FUSION commands
lasList <- list.files(points, pattern =".las", full.names = TRUE)

In [3]:
#write list of .las file to .txt file
las <- file.path(points, "lasList.txt")
if (file.exists(las)) file.remove(las)
d<-lapply(lasList, write, file.path(points, "lasList.txt"), append=T, ncolumns=1 )

[1] TRUE

In [4]:
lasList

[1] "D:/ForestMappingCIG/Data/Lidar/source/POINTS/ALL_POINTS/45122D4118.las"
[2] "D:/ForestMappingCIG/Data/Lidar/source/POINTS/ALL_POINTS/45122D4224.las"
[3] "D:/ForestMappingCIG/Data/Lidar/source/POINTS/ALL_POINTS/45122D4313.las"
[4] "D:/ForestMappingCIG/Data/Lidar/source/POINTS/ALL_POINTS/45122D4401.las"
[5] "D:/ForestMappingCIG/Data/Lidar/source/POINTS/ALL_POINTS/45122D4413.las"
[6] "D:/ForestMappingCIG/Data/Lidar/source/POINTS/ALL_POINTS/45122D4414.las"

### Clip lidar point clouds to FIA plot footprints

To manipulate and summary lidar data, we use the __[FUSION/LDV Lidar analysis and visualization](http://forsys.cfr.washington.edu/fusion/fusionlatest.html)__ software package developed by the Pacific Northwest Resaerch Station and University of Washington researchers. In order to run these tools, FUSION must be install on your local computer. 

First we clip the las files to the fia plot buffers that we created with the GIS notebook. 

In [17]:
#clip las files to FIA plot circles
system(paste(file.path("C:","Fusion", "PolyClipData.exe"), #path to FUSION executable on local computer
        paste("/multifile /shape:1,*"),
        paste(plots),
        paste(file.path(outDir, "clipLidar.las"), sep=""),
        paste(file.path(points, "lasList.txt"), sep="")), intern = T)

[1] "PolyClipData v1.60 (FUSION v3.50) (Built on Oct  9 2015 10:10:55) DEBUG"                                                                                                                                                        
 [2] " --Robert J. McGaughey--USDA Forest Service--Pacific Northwest Research Station"                                                                                                                                                
 [3] "Command line: C:\\Fusion\\POLYCL~1.EXE /multifile /shape:1,* D:/ForestMappingCIG/Data/FIA/FIA_plots.shp D:/ForestMappingCIG/Data/Lidar/output/clipLidar.las D:/ForestMappingCIG/Data/Lidar/source/POINTS/ALL_POINTS/lasList.txt"
 [4] "Run started: Fri Jul 06 09:59:59 2018"                                                                                                                                                                                          
 [5] "   D:/ForestMappingCIG/Data/Lidar/source/POINTS/ALL_POINTS/45122D4118.las: 37892714 points"                                                                                                                                     
 [6] "   D:/ForestMappingCIG/Data/Lidar/source/POINTS/ALL_POINTS/45122D4224.las: 20542100 points"                                                                                                                                     
 [7] "   D:/ForestMappingCIG/Data/Lidar/source/POINTS/ALL_POINTS/45122D4313.las: 15709351 points"                                                                                                                                     
 [8] "   D:/ForestMappingCIG/Data/Lidar/source/POINTS/ALL_POINTS/45122D4401.las: 19748596 points"                                                                                                                                     
 [9] "   D:/ForestMappingCIG/Data/Lidar/source/POINTS/ALL_POINTS/45122D4413.las: 15448670 points"                                                                                                                                     
[10] "   D:/ForestMappingCIG/Data/Lidar/source/POINTS/ALL_POINTS/45122D4414.las: 16938202 points"                                                                                                                                     
[11] "   Total: 126279633 points"                                                                                                                                                                                                     
[12] ""                                                                                                                                                                                                                               
[13] "Polygon 1:   ID:60634                              Pts:21   Parts:1"                                                                                                                                                            
[14] "   Extent: 822575.26,1338038.12   822935.26,1338398.12"                                                                                                                                                                         
[15] "   Part 1:   CW    Pts:21   1st pt:0"                                                                                                                                                                                           
[16] "Polygon 2:   ID:65759                              Pts:21   Parts:1"                                                                                                                                                            
[17] "   Extent: 806593.72,1356780.50   806953.72,1357140.50"                                                                                                                                                                         
[18] "   Part 1:   CW    Pts:21   1st pt:0"                               

### Calculate statistical predictor variables from clipped lidar data

Lidar predictor variables are computed using the FUSION *CloudMetrics* tool.  These metrics are computed using point elevations and intensity values.  This tool produces an output file containing a variety of statistical parameters describing the lidar data set. 

In [9]:
#create a list of .las files to use as input for FUSION tool
clippedList <- list.files(outDir, pattern=".las", full.names = TRUE)

In [10]:
#create a text file of the .las file list
las <- file.path(outDir, "clippedList.txt")
if (file.exists(las)) file.remove(las)
d<-lapply(clippedList, write, file.path(outDir, "clippedList.txt"), append=T, ncolumns=1)

In [17]:
#calculate cloud metrics 
system(paste(file.path("C:", "Fusion", "CloudMetrics.exe"),
     paste("/above:4.5"), 
     paste("/strata:[0,.15,1.37,5,10,20,30]"),
     paste("/id"),
     paste(file.path(outDir, "clippedList.txt")), 
     paste(file.path(outDir, "cloudmetrics.csv"))))

### Calculate topographic predictor variables from Bare Earth Terrian Model 

A 3 meter x 3 meter resolution raster grid containing bare earth eleveation values for our study area was used to calculate a variety of topographic attributes. The FUSION *TopoMetrics* tool is used to compute these metrics.  The output from this tool is a text file containing topographic indicators for our study area. 

In [21]:
#calculate topographic metrics
system(paste(file.path("C:", "Fusion", "TopoMetrics.exe"),
     paste(file.path(dem, "*.dtm")), 
     paste("3 3 45.439128 5"),
     paste(file.path(outDir, "topometrics.csv"))))

[1] "D:/ForestMappingCIG/Data/Lidar/source/POINTS/ALL_POINTS"